In [1]:
import streamlit as st
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from streamlit_folium import st_folium

ModuleNotFoundError: No module named 'streamlit'

In [ ]:
import streamlit as st
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from streamlit_folium import st_folium

# -------------------------
# Cargar datos
# -------------------------
@st.cache_data
def load_data():
    # Ajusta estas rutas a tus datasets
    hospitals = pd.read_csv("data/hospitals.csv")  # MINSA – IPRESS
    pop_centers = pd.read_csv("data/pop_centers.csv")

    # Convertir a GeoDataFrame
    gdf_hospitals = gpd.GeoDataFrame(
        hospitals,
        geometry=gpd.points_from_xy(hospitals["longitude"], hospitals["latitude"]),
        crs="EPSG:4326"
    )

    return gdf_hospitals, pop_centers

hospitals_gdf, pop_centers = load_data()

# -------------------------
# Configuración principal
# -------------------------
st.set_page_config(page_title="Peru Hospitals Analysis", layout="wide")

st.title("🏥 Geospatial Analysis of Public Hospitals in Peru")

# Tabs
tab1, tab2, tab3 = st.tabs(["🗂️ Data Description", "🗺️ Static Maps & Department Analysis", "🌍 Dynamic Maps"])

# -------------------------
# TAB 1: Data Description
# -------------------------
with tab1:
    st.header("📊 Data Description")

    st.markdown("""
    **Unit of Analysis:** Operational public hospitals in Peru  
    **Data Sources:** MINSA – IPRESS (operational subset), Population Centers  
    **Filtering rules:** Only operational hospitals with valid latitude/longitude  
    """)

    st.subheader("Preview of Hospitals Data")
    st.dataframe(hospitals_gdf.head())

    st.map(hospitals_gdf, latitude="latitude", longitude="longitude")

# -------------------------
# TAB 2: Static Maps & Department Analysis
# -------------------------
with tab2:
    st.header("🗺️ Static Maps & Department Analysis")

    # Choropleth by department (ejemplo)
    st.subheader("Department-level Hospital Distribution")

    # Suponiendo que tienes un shapefile de departamentos
    departments = gpd.read_file("data/peru_departments.shp")
    dept_summary = hospitals_gdf.groupby("department").size().reset_index(name="hospital_count")
    departments = departments.merge(dept_summary, left_on="NOMBDEP", right_on="department", how="left").fillna(0)

    # Plot static choropleth with GeoPandas
    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    departments.plot(column="hospital_count", cmap="OrRd", legend=True, ax=ax, edgecolor="black")
    ax.set_title("Hospitals per Department in Peru", fontsize=14)
    st.pyplot(fig)

    # Table
    st.subheader("Summary Table")
    st.dataframe(dept_summary.sort_values("hospital_count", ascending=False))

    # Bar chart
    st.subheader("Bar Chart")
    fig, ax = plt.subplots(figsize=(12, 6))
    dept_summary.sort_values("hospital_count", ascending=False).plot(
        kind="bar", x="department", y="hospital_count", ax=ax, color="steelblue"
    )
    plt.xticks(rotation=90)
    st.pyplot(fig)

# -------------------------
# TAB 3: Dynamic Maps
# -------------------------
with tab3:
    st.header("🌍 Dynamic Maps")

    # National Folium choropleth + markers
    st.subheader("National Folium Choropleth")

    # Base map
    m = folium.Map(location=[-9.19, -75.0152], zoom_start=5)

    # Choropleth by department
    folium.Choropleth(
        geo_data=departments.to_json(),
        data=dept_summary,
        columns=["department", "hospital_count"],
        key_on="feature.properties.NOMBDEP",
        fill_color="YlOrRd",
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name="Hospitals per Department"
    ).add_to(m)

    # Hospital markers
    for _, row in hospitals_gdf.iterrows():
        folium.CircleMarker(
            location=[row["latitude"], row["longitude"]],
            radius=2,
            popup=row["name"] if "name" in hospitals_gdf.columns else "Hospital",
            color="blue",
            fill=True,
            fill_opacity=0.6
        ).add_to(m)

    st_folium(m, width=900, height=500)

    # Proximity maps for Lima & Loreto
    st.subheader("Proximity Analysis: Lima & Loreto")
    for region in ["Lima", "Loreto"]:
        st.markdown(f"**{region}**")
        region_data = hospitals_gdf[hospitals_gdf["department"] == region]

        m_region = folium.Map(location=[region_data.latitude.mean(), region_data.longitude.mean()], zoom_start=7)
        for _, row in region_data.iterrows():
            folium.Marker(
                location=[row["latitude"], row["longitude"]],
                popup=row["name"] if "name" in hospitals_gdf.columns else "Hospital"
            ).add_to(m_region)

        st_folium(m_region, width=900, height=500)
